# Momentum Contrast for Unsupervised Visual Representation Learning  
- paper review

## Abstract 
- 우리는 unsupervised visual representation을 위한 Momentum Contrast(MoCo)를 제안한다  
- contrastive learning의 관점에서, 우리는 queue와 moving-averaged encoder를 통해 dynamic dictionary를 구축한다  
- 이를 통해 contrastive unsupervised learning이 가능하게끔 크고 일관된 dictionary를 구축할 수 있다  
- MoCo는 7개의 downstream task에서 supervised pre-training counterpart의 성능을 크게 능가한다  
- 이는 vision task에서 unsupervised와 supervised의 성능의 gap이 많이 좁혀졌음을 보여준다

## Introduction  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/figure1.png?raw=true" width=500>

- unsupervised representation learning은 NLP task에서 크게 성공을 했으며 예로 GPT, BERT 등이 있다  
- language task는 tokenized dictionaries를 위해 각각 별개의 signal space(words, sub-word units 등)를 가지고 있다  
- 그에 반해 vision task에서는 raw signal이 continuous, high-dimensional space에 있으며 인간의 커뮤니케이션을 위해 구조화할 수 없다  
- dictionary에서 "keys"(tokens)는 data로부터 샘플링되며 (images or patches) encoder network에 의해 표현된다  
- unsupervised learning은 dictionary look-up을 위해 encoder를 훈련하며 encoded "query"는 해당 키와 유사해야 하며 다른 것과는 달라야 한다  
- 이러한 관점으로부터, 우리는 학습하는 동안 진화하면서 크고 일관적인 dictionary를 구축하기 위해 가설을 세운다  
- 우리는 contrastive loss와 unsupervised learning을 위해 크고 일관적인 dictionary를 구축하는 방법으로 MoCo를 제시한다  
- 우리는 dictionary를 데이터 샘플의 queue로 유지한다  
- 최근 mini-batch의 encoded resentation은 enqueued되며 가장 오래된 것은 dequeued된다  
- queue는 mini-batch size로부터 dictionary size를 분리시키며 size를 키우도록 유도한다  
- MoCo는 dynamic dictionary를 구축하기 위한 mechanism이며 다양한 pretext task로 사용될 수 있다  
- MoCo는 ImageNet dataset에서 linear classification에 대해 경쟁력 있는 성능을 보여준다  
- MoCo는 ImageNet 또는 다양한 large dataset으로 사전 훈련하면 real-world, 엄청난 image scale에서도 잘 작동한다  
- 이러한 결과는 MoCo가 supervised와 unsupervised의 성능 차이의 gap을 크게 줄여줌을 보여준다  


## Method 
### Contrastive Learning as Dictionary Look-up 
- encoding된 query $q$와 encoding된 sample set $\left\{k_0, k_1, k_2,\cdots\right\}$을 고려하자  
- dictionary에 $q$와 일치하는 하나의 key($k_+$로 표시)가 있다고 가정한다  
- contrastive loss는 q가 positive key $k_+$와 유사하고 다른 keys에는 유사하지 않을 때 적은 값을 가지는 함수이다  
- 우리 loss function은 dot product에 의해 측정되는 유사도로 contrastive loss function의 형태가 정해지며 InfoNCE라고 부른다  
$$\mathcal{L}_q=-log\frac{exp\left(q\cdot k_+/\tau\right)}{\sum_{i=1}^K exp\left(q\cdot k_i/\tau\right)}$$  
- 이때 $\tau$는 temperature hyperparameter이다  
- 직관적으로 위의 loss function은 $q$를 $k_+$로 분류하는 $\left(K+1\right)$-way softmax-based function이다  
- contrastive loss는 query와 key를 표현하는 encoder network를 훈련하기 위해 unsupervised objective function의 역할을 한다  
- 일반적으로, query representation은 $q=f_q\left(x^q\right)$이며 이때 $f_q$는 encoder network, $x^q$는 query sample이다  
- input $x^q$와 $x^k$는 images, patches 또는 patch의 set을 포함하는 context이다  

### Momentum Contrast 
- dictionary는 key가 랜덤하게 샘플링되고 key encoder가 훈련하면서 진화한다는 점에서 dynamic하다  
- 우리의 가설은 많은 negative samples를 커버하는 large dictionary에 의해 good features를 얻을 수 있도록 잘 학습이 될 수 있는 반면 dictionary keys에 대한 encoder는 진화에도 불구하고 가능한 일관되게 유지된다는 것이다  

**Dictionary as a queue**  

- 우리의 접근법의 핵심은 dictionary를 data samples의 queue로 유지하는 것이다    
- 이를 통해 우리는 이전의 mini-batches에서 encoded keys를 재사용할 수 있다  
- queue의 introduction은 dictionary size를 mini-batch size와 분리한다  
- 우리의 dictionary size는 mini-batch size보다 크게 잡아도 되며 hyperparameter를 유연하고 독립적으로 설정할 수 있다  
- 최근 mini-batch는 dictionary로 enqueued되고 가장 오래된 mini-batch는 제거된다  

**Momentum update**  
- 우리는 key representation consistency를 줄이는 encoder가 빠르게 변화함으로 인해 실패가 일어난다고 가설을 세웠다  
- 그래서 우리는 위의 issue를 해결하기 위해 momentum update를 제안한다  

$$\theta_k \leftarrow m\theta_k + \left(1-m\right)\theta_q$$  
- momentum update form이며 $m\in\left[0,1\right)$은 momentum coefficient이다  
- 오직 parameters $\theta_q$가 back-propagation으로 업데이트 된다  
- 위 식은 $\theta_k$가 $\theta_q$보다 부드럽게 업데이트되도록 해준다  
- 그 결과로 queue에 있는 keys는 각각 다른 encoders들에 의해 encoding되지만 encoders 사이의 차이는 작게 만들 수 있다  
- 우리의 실험에서, large momentum $(m=0.999)$가 smaller value $(m=0.9)$보다 더 좋은 성능을 보여줬다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/figure2.png?raw=true" width=800>
  
**Relations to previous mechanisms**  
- 우리는 figure 2에서 일반적인 다른 mechanism과 비교했다  
- 이들은 dictionary size와 consistency 측면에서 서로 다른 성질을 보여준다  
- 먼저 **end-to-end**는 back-propagation에 의해 업데이트하는 mechanism이다  
- 이는 최근 mini-batch에서 샘플을 추출해 dictionary로 사용하며 keys가 일관적으로 encoding된다  
- 그러나 dictionary size는 mini-batch size와 연결되므로 GPU memory size에 따라 한계가 존재한다  
- large mini-batch의 경우 optimization에 있어서 어려움이 있다  
- 또 다른 mechanism으로 **memory bank**가 있다  
- 이는 dataset에서 모든 samples의 representation을 포함하고 있다  
- 각 mini-batch에 대한 dictionary는 memory bank에서 랜덤하게 추출된다  
- 이때 back-propagation을 진행하지 않아 large dictionary size를 사용할 수 있다  
- 그러나 memory bank에서 sample의 representation는 마지막에 본 것만 업데이트된다  
- 즉, 임의로 몇 개의 sample들만 골라 dictionary를 구성하고 query에 해당하는 sample들로 memory bank를 업데이트하면서 학습을 진행한다  
- 따라서 dictionary가 일관적이지 않은 형태를 가지게 되며 학습이 제대로 되지 않는 경우가 생긴다  
- 이를 해결하기 위해 momentum update를 적용했다  
- 그러나 이는 encoder가 아닌 sample의 representation(key)을 업데이트하기 때문에 마찬가지로 일관적이지 않은 형태를 띠게 된다  
- 반면, MoCo는 위 두 방법과 달리 모든 샘플이 아닌 queue를 이용한 dictionary를 활용하기 때문에 충분한 양의 negative sample로 학습할 수 있다  
- 또한 memory-efficient하며 large scale data에 대해서도 학습할 수 있다는 장점이 있다  

### Pretext Task 
- 이 논문에서 집중하고 있는 것은 새로운 pretext task를 디자인 하는 것이 아니라 간단한 instance discrimination task를 따르는 것이다  
- 우리는 같은 이미지에서 나온 query와 key로 이루어진 positive pair, 다른 이미지에서 나온 negative pair를 고려한다  
- data augmentation을 적용해 같은 이미지에 대해서는 random views의 positive pair를 얻는다  
- queries와 keys는 각각 encoders를 거치고 나면 $f_q$와 $f_k$를 얻는다  
- 이때 encoder는 CNN model로도 할 수 있다  

**Technical details**  
- 우리는 encoder network로 ResNet을 적용하며 마지막 fc layer는 128-dimension으로 고정시킨다  
- 그리고 output vector는 L2-norm으로 normalize를 해준다  
- 이는 query 또는 key의 representation이 된다  
- 이때 temperature $\tau$는 0.07로 설정한다  
- data augmentation  
    - random resized  
    - $224\times 224$ crop  
    - random color jitter  
    - random horizontal flip  
    - random grayscale  
    
**Shuffling BN**  
- encoders $f_q$와 $f_k$ 모두 ResNet에서 BN을 적용한다  
- 실험을 한 결과, 우리는 BN이 good representation을 학습하는 것을 방해한다는 걸 확인했다  
- 모델이 pretext task를 "cheat"하는 것 처럼 보이며 low-loss solution을 쉽게 찾음을 의미한다  
- 이는 samples 사이의 intra-batch communication이 information을 누락하기 때문일 수 있다  
- 우리는 이 문제를 shuffling BN을 통해 해결했다  
- multi GPUs로 훈련을 시키고 각 GPU에서 독립적으로 samples에 BN을 적용한다  
- key encoder $f_k$에 대해서, 우리는 GPUs에 배포하기 전에 최근 mini-batch의 sample order를 shuffling을 통해 수행한다  
- 이때, query encoder $f_q$는 shuffling을 하지 않는다  
- 이렇게 해결함으로써 BN으로 학습이 더 잘되게 할 수 있다  

## Experiments 
**ImageNet-1M**  
- label이 학습에 이용되지 않기 때문에 우리는 이미지 갯수를 센다  

**Instagram-1B**  
- instagram에서 추출한 약 940M개의 이미지로 이루어져 있으며 약 1500개의 hashtags를 가지고 있다  

**Training**  
- optimizer: SGD momentum 0.9. 
- weight decay: 0.0001. 
- For IN-1M, we use a mini-batch size of 256 in 8 GPUs  
- initial learning rate: 0.3  
- We train for 200 epochs with the lr multiplied by 0.1 at 120 and 160 epochs  
- training time: 53 hours for ResNet-50  
- For IG-1B, we use a mini-batch size of 1024 in 64 GPUs  
- We use a learning rate of 0.12 which is exponentially decayed by 0.9x after every 62.5k iterations  
- We train for 1.25M iterations  
- training time: 6 days for ResNet-50  

### Linear Classification Protocol 
- 이 section에서는 ImageNet-1M에서 unsupervised pre-training을 수행한다  
- 우리는 features를 freeze하고 ResNet에서 GAP 이후에 classifier를 적용해 100 epochs로 학습한다  
- best combination of hyperparameters를 위해 grid search를 이용하고 최적의 initial lr은 30, weight decay는 0이다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/figure3.png?raw=true">

**Ablation: contrastive loss mechanisms**  
- InfoNCE loss function을 사용해 각 mechanism의 결과를 figure 3에 나타냈다  
- 전체적으로 $K$가 클수록 좋은 성능을 보여준다  
- 먼저 **end-to-end** mechanism은 K가 작을 때 MoCo와 비슷한 성능을 보여준다  
- 그러나 dictionary size가 mini-batch size 때문에 제한이 있는 것이 단점이다  
- 더 근본적으로 large mini-batch training에 문제가 있는데, 여기서 linear lr scaling rule를 사용해야 한다는 것을 발견했다  
- 그래서 이 rule 없이는 정확도가 2% 정도 떨어진다 (1024 mini-batch)  
- 또한, larger mini-batch로는 optimizing이 어려워진다  
- **memory bank** mechanism은 larger dictionary size를 사용할 수 있다  
- 그러나 MoCo보다 2.6% 정도 정확도가 떨어진다  
- momery bank에 있는 keys들은 past epoch 전반에 걸쳐 다른 encoders로부터 온 것이며 이들은 일관적이지 않다는 문제가 있다  

**Ablation: momentum**  
- 다음 table은 각 MoCo momentum values로 ResNet-50 accuracy를 측정한 것이다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/sub_table.png?raw=true" width=450>

- 이는 m이 0.99~0.9999일 때 상당히 잘 수행된다는 것을 보여준다  
- m이 너무 작을 때 ($m=0.9$) accuracy가 상당히 떨어지는 것을 볼 수 있고 $m=0$일 때는 loss가 진동하면서 결국 수렴에 실패한다  
- 이는 consistent dictionary를 구축함에 있어 motivation을 얻게 해줬다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table1.png?raw=true">

**Comparison with previous results**  
- 이전 unsupervised learning method들은 model size에 따라 성능이 많이 달라졌다  
- table 1에서 그 결과를 확인할 수 있다  
- ResNet-50에서 다양한 사이즈를 적용해 2x, 4x (more channels)로도 실험해봤다  
- 그리고 $K=65536$, $m=0.999$로 설정했다  
- R50의 MoCo는 60.6%의 accuracy를 달성하며 경쟁력 있는 성능을 보여줬다  
- 특히 비슷한 모델 사이즈(~24M)에서 가장 좋은 성능을 보여준다  
- 또한 MoCo는 모델 사이즈가 클수록 더 좋은 성능을 보여준다 (68.6% with R50w4x)  

### Transferring Features  
- unsupervised learinng의 목표는 transferrable한 features를 학습시키는 것이다  
- ImageNet supervised pre-training은 downstream task의 fine-tuning에서 가장 영향력 있는 method이다  
- 우리는 ImageNet supervised pre-training과 MoCo를 비교하며, 다양한 task에 transfer learning을 적용해 성능을 살펴본다  
- 이때 dataset으로는 COCO, PASCAL VOC 등을 사용한다  
- 먼저, normalization과 schedules 2가지 이슈에 대해 의논한다  

**Normalization**  
- unsupervised pre-training에 의해 생성된 features는 supervised features와 distribution이 다를 수 있다  
- 그러나 downstream task에서는 hyperparameter를 supervised pre-training 기준에 맞춰 설정한다  
- 이를 완화하기 위해, 우리는 fine-tuning 동안 feature normalization을 적용한다  
- affine layer에 의해 freeze되는 대신 훈련된 BN을 fune-tuning한다  
- 이를 적용함으로써 MoCo는 supervised method와 같은 hyperparameters를 설정해도 문제가 없게 되었다  

**Schedules**  
- 우리 목적은 transferability한 features를 얻는 것이기 때문에 다양한 schedules를 적용해 실험해본다  
- fune-tuning에서 MoCo는 supervised와 같은 schedule을 사용한다  
- 즉, 종합하면 우리 fine-tuning은 supervised pre-training setting과 동일하게 사용하며  
- 이는 MoCo에 disadvantage가 되지만 그럼에도 불구하고 MoCo는 경쟁력 있는 성능을 보여준다  

#### PASCAL VOC Object Detection 
**Setup**  
- detector로 backbone이 R50-dilated-C5 or R50-C4인 Faster R-CNN을 사용한다  
- 학습할 때 image scale은 $(480, 800)$, inference 할 때는 800으로 한다  
- 전반적으로 setup은 supervised pre-training baseline과 동일하다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table2.png?raw=true">

**Ablation: backbones**  
- table 2에서 각 성능을 보여주고 있으며 역시 MoCo가 좋은 성능을 보여주고 있다  
- supervised counterpart보다 0.9, 3.7, 4.9 AP 더 좋은 성능을 보여준다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table3.png?raw=true">

**Ablation: contrastive loss mechanisms**  
- constrative learning에서 MoCo mechanism만을 사용해 기여도를 측정하기 위해 우리는 end-to-end or memory bank mechanism과 비교한다  
- table 3에서 다른 mechanism에 비해 MoCo가 월등하게 성능이 좋음을 볼 수 있다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table4.png?raw=true">

**Comparision with previous results**  
- table 4에서 다른 unsupervised method과 비교한 결과를 볼 수 있으며 역시 MoCo가 가장 성능이 좋다  

#### COCO Object Detection and Segmentation 
**Setup**  
- model은 FPN or C4 backbone의 Mask R-CNN을 사용한다  
- 학습할 때 image scale은 $(640, 800)$, inference에서는 800을 사용한다  
- schedule은 1x, 2x를 사용했다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table5.png?raw=true">

**Results**  
- table 5에서 각 성능을 보여주고 있다  
- 1x schedule을 사용했을 때, 좋지 않은 성능을 보여주고 있다  
- 반면 2x schedule에서는 MoCo가 supervised method들보다 모든 metric에서 좋은 성능을 보여주고 있다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/MoCo/figure/table6.png?raw=true">

#### More Downstream Tasks  
**Summary**  
- 종합적으로, MoCo는 7개의 downstream task에서 ImageNet supervised pre-training counterpart보다 좋은 성능을 보여준다  
- Cityscapes instance segmentations에서는 적당한 성능을 보여주며 VOC semantic segmentation에서는 그닥 좋지 못한 성능을 보여줬다  
- 다시 살펴보면 모든 task에서 MoCo는 IN-1M보다 IG-1B로 pre-training할 때 조금 더 좋은 성능을 보여준다  
- 이는 MoCo가 large scale dataset에서 잘 작동될 것임을 의미한다  

## Discussion and Conclusion. 
- 우리는 다양한 cv task에서 긍정적인 결과를 보여주는 unsupervised learning을 보여줬다  
- IN-1M에서 IG-1B로의 MoCo의 개선은 뚜렷한 일관성을 보여주지만 그렇게 크지는 않다  
- 이는 large dataset에서도 제대로 활용되지 않을 수 있음을 암시한다   
- 우리는 MoCo가 contrastive learning을 포함해 다른 pretext tasks에서 잘 활용되기를 바란다